In [1]:
from datetime import date, datetime
import pandas as pd
import subprocess 
from subprocess import Popen
import time
import io, contextlib
from zipfile import ZipFile
from databases import Bawm, Crm, PosicaoDm1,PosicaoAdministrador,Boletador,BDS
from emailer import Email, EmailLer
from funcoes_datas import FuncoesDatas
from filemanager import FileMgmt
from datetime import date, datetime, timedelta
import win32com.client as win32
from databases import Bawm, Crm,PosicaoDm1
from zipfile import ZipFile
import os
import calendar
import numpy as np
import ambiente
from funcoes_datas import FuncoesDatas
from objetos import Fundo
from pretty_html_table import build_table

[base_dados.py directory]: C:\Temp\Foundation
[databases.py directory]: C:\Temp\Foundation


In [19]:
#Puxa as Classes do Data Base
bawm = Bawm()
datas =  FuncoesDatas()
hoje = datas.hoje()
posicao = PosicaoDm1()


In [20]:
posicao = posicao.__posicao_fundos_all__()

In [21]:
cadastro = bawm.po_cadastro()

In [22]:
#Verificar por emissor 20%

posicao_emissor = posicao[posicao['NomeEmissor'].notnull()]
posicao_emissor = posicao_emissor[posicao_emissor['NomeEmissor']!=None]
posicao_emissor = posicao_emissor[['NomeContaCRM','TipoProduto','Classe','NomeEmissor','PesoFinal']]
posicao_emissor = posicao_emissor.groupby(['NomeContaCRM','TipoProduto','NomeEmissor']).sum().reset_index().sort_values(by=['PesoFinal'],ascending=False).reset_index(drop=True)
posicao_emissor['PesoFinal'] = posicao_emissor['PesoFinal']*100
posicao_emissor = posicao_emissor[(posicao_emissor['PesoFinal']<100)&(posicao_emissor['PesoFinal']>1)]

In [23]:
#Percentual por emissor no exterior - 20%

posicao_exterior = posicao[posicao['Classe'].str.contains('Internacional')]
posicao_exterior = posicao_exterior[['NomeContaCRM','NomeProduto','TipoProduto','PesoFinal']].groupby(['NomeContaCRM','NomeProduto']).sum().reset_index().sort_values(by=['PesoFinal'],ascending=False).reset_index(drop=True)
posicao_exterior['PesoFinal'] = posicao_exterior['PesoFinal']*100
posicao_exterior = posicao_exterior[(posicao_exterior['PesoFinal']<100)&(posicao_exterior['PesoFinal']>1)]

In [24]:
#Percentual do somatório de cotas dos FIDCs + FIPs - 50%
# Percentual por emissor 20%

posicao['NomeProduto'] = posicao['NomeProduto'].str.lower()

posicao_fip = posicao[posicao['NomeProduto'].str.contains('fip')]
posicao_fip_emissor = posicao_fip[['NomeContaCRM','PesoFinal','NomeProduto']].groupby(['NomeContaCRM','NomeProduto']).sum().reset_index().sort_values(by=['PesoFinal'],ascending=False).reset_index(drop=True)
posicao_fip_emissor['PesoFinal'] = posicao_fip_emissor['PesoFinal']*100

posicao_fidc = posicao[posicao['NomeProduto'].str.contains('fidc')]
posicao_fidc_emissor = posicao_fidc[['NomeContaCRM','PesoFinal','NomeProduto']].groupby(['NomeContaCRM','NomeProduto']).sum().reset_index().sort_values(by=['PesoFinal'],ascending=False).reset_index(drop=True)
posicao_fidc_emissor['PesoFinal'] = posicao_fidc_emissor['PesoFinal']*100

posicao_fip_fidc = pd.concat([posicao_fip,posicao_fidc])
posicao_fip_fidc = posicao_fip_fidc[['NomeContaCRM','PesoFinal']].groupby(['NomeContaCRM']).sum().reset_index().sort_values(by=['PesoFinal'],ascending=False).reset_index(drop=True)
posicao_fip_fidc['PesoFinal'] = posicao_fip_fidc['PesoFinal']*100
posicao_fip_fidc = posicao_fip_fidc[(posicao_fip_fidc['PesoFinal']<100)&(posicao_fip_fidc['PesoFinal']>1)]

In [30]:
posicao_fidc_emissor

,NomeContaCRM,NomeProduto,PesoFinal
0,_Fiducia Master FIM CrPr IE,multi asset fidc multissetorial - sr 11 (brl),37.046803
1,_Black Hawk FIM CrPr IE,vega fidc - sb (brl),17.043140
2,_Charlotte FIM CP IE,"sb fidc multissetorial - 5s1e sr cdi+4,50% 16/...",11.959159
3,_Carmel JBFO FIM CrPr IE,"fidc cartao de compra supplier - mz i cdi+3,00...",9.555735
4,_Pedra Negra RFP FIM CP IE,"sifra energy podium fidc - 1s2e sr cdi+2,92% 1...",9.300529
...,...,...,...
1709,_Cheers FIC FIM CrPr IE,"sav nexoos fidc - 2s1e cdi+4,50% 22/fev/26 (brl)",-0.020870
1710,_Arten FIM CrPr IE,green fidc solar gd - fidc socioambiental - en...,-0.228223
1711,_Arten FIM CrPr IE,green fidc solar gd - fidc socioambiental - en...,-0.379651
1712,_Arten FIM CrPr IE,green fidc solar gd - fidc socioambiental - en...,-0.425299


In [25]:
#Percentual dos fundos ilimitados (dada nossa alocação atual) - 40%

dicio_classes_ilimitado = {'R Fixa Pós':0.37,'R Fixa Infl':0.33,'R Variável':0.3,'Investimento no Exterior':0.5,'Outros':0,
                          'F Exclusivo':0,'P. Equity':1,'Alternativos':0.3,'Crédito Alternativo':1,'Ajuste':0,'RV Internacional':1,
                           'Cambial':1,'R Fixa Pré':0,'Infraestrutura Ilíquidos':1,'Real Estate':0, 'RV Global Hedged':1,'F Exclusivo Ações':1,
                           'RF Internacional':0,'RF Distressed':0}

posicao_ilimitado = posicao[['NomeContaCRM','Classe','SubClasse','PesoFinal']].groupby(['NomeContaCRM','Classe','SubClasse']).sum().reset_index().sort_values(by=['PesoFinal'],ascending=False).reset_index(drop=True)
posicao_ilimitado['PesoFinal'] = posicao_ilimitado['PesoFinal'] * 100
posicao_ilimitado['% Ilimitado'] = posicao_ilimitado['Classe'].apply(lambda x : dicio_classes_ilimitado[x])
posicao_ilimitado['%F.I'] = posicao_ilimitado['% Ilimitado'] * posicao_ilimitado['PesoFinal']
posicao_ilimitado = posicao_ilimitado[(posicao_ilimitado['PesoFinal']<100)&(posicao_ilimitado['PesoFinal']>1)]

In [26]:
#Limite por alocação - 25%

limite_alocacao = posicao[['NomeContaCRM','NomeProduto','PesoFinal']].groupby(['NomeContaCRM','NomeProduto']).sum().reset_index().sort_values(by=['PesoFinal'],ascending=False).reset_index(drop=True)
limite_alocacao['PesoFinal'] = limite_alocacao['PesoFinal']*100
limite_alocacao = limite_alocacao[(limite_alocacao['PesoFinal']<100)&(limite_alocacao['PesoFinal']>1)]

In [32]:
#Gerando excel para e-mail:
with pd.ExcelWriter('C:/Temp/Foundation/Analise_Limitadosv2.xlsx') as writer:  
    posicao_emissor.to_excel(writer, sheet_name='emissor')
    posicao_exterior.to_excel(writer, sheet_name='emissor_exterior')
    posicao_fip_emissor.to_excel(writer, sheet_name='emissor_fip')
    posicao_fidc_emissor.to_excel(writer, sheet_name='emissor_fidc')
    posicao_fip_fidc.to_excel(writer, sheet_name='emissor_fidc_fip')
    posicao_ilimitado.to_excel(writer, sheet_name='posicao_ilimitado')
    limite_alocacao.to_excel(writer, sheet_name='limite_alocacao')